<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/spell/j_spell_check%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본		    G_inc_again_so_model.pt
 산업분류자동화    'My Drive'
'Colab Notebooks'   Screenshot_20220401-001411_Chrome.jpg


In [27]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/산업분류자동화/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [28]:
df=df[(df['digit_1'] =='J')] #조건에 맞는 데이터 출력

In [29]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000065,J,58,582,사무실에서,고객의뢰로,시스템소프트웨어개발및공급
1,id_0000107,J,58,582,사업장에서,고객의뢰를받아,무인주차관리시스템개발공급
2,id_0000157,J,59,592,인터넷으로,디지털음원,음반제작유통
3,id_0000276,J,58,582,사무실에서,의뢰받아,소프트웨어개발
4,id_0000338,J,58,582,사무실에서,고객의뢰로,시스템소프트웨어개발및공급
...,...,...,...,...,...,...,...
10857,id_0999684,J,58,582,사무실에서,고객의뢰로,시스템소프트웨어개발
10858,id_0999872,J,62,620,사무실에서,의뢰받아,소프트웨어개발
10859,id_0999914,J,59,592,NaN,녹음시설운영업,NaN
10860,id_0999959,J,58,582,사무실에서,고객의뢰로,시스템 소프트웨어개발


In [30]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000065,J,58,582,사무실에서 고객의뢰로 시스템소프트웨어개발및공급
1,id_0000107,J,58,582,사업장에서 고객의뢰를받아 무인주차관리시스템개발공급
2,id_0000157,J,59,592,인터넷으로 디지털음원 음반제작유통
3,id_0000276,J,58,582,사무실에서 의뢰받아 소프트웨어개발
4,id_0000338,J,58,582,사무실에서 고객의뢰로 시스템소프트웨어개발및공급
...,...,...,...,...,...
10857,id_0999684,J,58,582,사무실에서 고객의뢰로 시스템소프트웨어개발
10858,id_0999872,J,62,620,사무실에서 의뢰받아 소프트웨어개발
10859,id_0999914,J,59,592,녹음시설운영업
10860,id_0999959,J,58,582,사무실에서 고객의뢰로 시스템 소프트웨어개발


In [31]:
df['digit_3'].value_counts()

582    3771
620    1919
581    1548
591    1250
611     796
631     456
639     369
612     317
592     230
602     186
601      20
Name: digit_3, dtype: int64

## 맞춤법 교정

In [24]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-i07xs6hx
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-i07xs6hx


In [32]:
from hanspell import spell_checker

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)

df['text']=pd.DataFrame(temp_result)  

In [33]:
df.to_csv('df_J_spellcheck.csv', index=False, encoding='utf-8-sig')